In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import lightning.pytorch as pl
import PPIGraph

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# Load dataset

In [7]:
from torch_geometric.loader import DataLoader

# Define condition mapping
condition_mapping = {'HCC827Ctrl': 0, 'HCC827Osim': 1}

# Load graph dataset and process if neede
graph_path = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'graphs' 

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


Processing...


  0%|          | 0/2305 [00:00<?, ?it/s]

c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\torch_geometric\utils\convert.py:249: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  data[key] = torch.tensor(value)
Done!


In [8]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset(2305):
Number of graphs: 2305
Number of features: 5
Number of classes: 2
Train set: 1107, test set: 922, val set: 276
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 42956], pos=[3376, 3], labels=[3376, 5], nuclei=[3376], weight=[42956], condition=[32], fov=[32], id=[32], train_mask=[3376], test_mask=[3376], x=[3376, 5], y=[32], edge_weight=[42956], name=[32], batch=[3376], ptr=[33])



In [9]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset(2305):
Number of graphs: 2305
Number of features: 5
Number of classes: 2
Train set: 1107, test set: 922, val set: 276
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 38672], pos=[3093, 3], labels=[3093, 5], nuclei=[3093], weight=[38672], condition=[32], fov=[32], id=[32], train_mask=[3093], test_mask=[3093], x=[3093, 5], y=[32], edge_weight=[38672], name=[32], batch=[3093], ptr=[33])



# K-fold on filter dataset

In [10]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [12]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 70

graph_path = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'graphs' 

dataset = PPIGraph.GraphDataset(graph_path, 'raw', 'pt', condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [13]:
len(dataset_filtered)

1595

In [14]:
dataset_name = '5PPI_3D'
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' /'OCT Cell Culture' / '3D_Whole' /"saved_models" / dataset_name / f"Graph_GNNs_{condition}" 
project_name = f'PLA_01122024_{dataset_name}_{condition}'

In [15]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)
epochs = 65

# NUM_LAYERS = 2
# HIDDEN_CHANNELS = 16
# pools = ['mean', 'max', 'sum', 'attention', 'attention2']
# num_layers = [2,3,4]
# hiddens = [16, 32, 64]


# # model = 'GAT'
# model = 'GINConv'
params = [[3,32,'mean','MLP'], 
[3,32,'attention2','GraphConv'],
[2,64,'attention2','GCN'],
[3,64,'attention','GAT_norm'],
[2,16,'attention','GINConv_norm']]

Global seed set to 42


In [16]:
params

[[3, 32, 'mean', 'MLP'],
 [3, 32, 'attention2', 'GraphConv'],
 [2, 64, 'attention2', 'GCN'],
 [3, 64, 'attention', 'GAT_norm'],
 [2, 16, 'attention', 'GINConv_norm']]

In [17]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    for NUM_LAYERS, HIDDEN_CHANNELS, pool, model in params:
            # Path to the folder where the pretrained models are saved
        # CHECKPOINT_PATH = checkpoint_folder / f'GAT_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue

        # Run training
        run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}', 
                        group=f'{model}_{pool}', 
                        # mode="disabled"
                        )
        PPIGraph.train_graph_classifier_kfold(model, 
                                                train_subset, 
                                                val_subset, 
                                                dataset, 
                                                CHECKPOINT_PATH, 
                                                AVAIL_GPUS, 
                                                in_channels=5,
                                                hidden_channels=HIDDEN_CHANNELS, 
                                                out_channels = HIDDEN_CHANNELS,
                                                num_layers=NUM_LAYERS, 
                                                epochs=epochs,
                                                embedding=False,
                                                batch_size=128,
                                                graph_pooling=pool)
        run.finish()


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\pytorch_lightning\loggers\wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42


Using MLP


You are using a CUDA device ('NVIDIA RTX A2000 12GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\23_PLA_revision\data\OCT Cell Culture\3D_Whole\saved_models\5PPI_3D\Graph_GNNs_Kfold\MLP_3_32_onehot_0\mean\GraphLevelMLP exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type             | Params
-------------------------------------------------
0 | x_embedding | Identity         | 0     
1 | model       | MLP

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▄▄▄▅▇▇▇▇▇▇▆▇▇▆▇▇▇▇█▇▇▇▇▇█▇▇█▇▇▇▇▇▇▇▇▇▇▇
train_auc,▁▁▁▂▄▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇█▇███████▇████▇█▇█
train_f1,▅▁▁▁▂▇█▇█▇█▇▇▇▇▇██▇█▇▇█▇▇█▇▇█▇█▇▇█▇▇▇▇▇▇
train_loss_epoch,█▇▆▆▅▂▂▂▂▂▁▃▂▂▂▂▁▁▂▂▂▁▁▂▁▂▁▁▁▂▂▂▂▁▂▂▂▂▂▂
train_loss_step,███▆█▆▄▅▄▄▄▄▂▄▃▄▁▄▅▆▂▂▃▃▁▂▂▂▄▄▆▂▂▇▄▄▄▃▂▅
trainer/global_step,▁▁▁▂▂▁▂▂▂▃▂▃▃▂▂▄▄▄▂▂▅▅▂▅▂▆▆▂▆▆▃▃▃▇▇▃▃██▃
val_acc,▁▁▁▁▃█▇▇▇▇██▆▆▇▇▇███▇▇▇▇█▇▇▇▇▇▇█▇▇▇▇▇▇▆▇
val_auc,▁▇██████████████████████████████████████
val_f1,▁▁▁▁▃█▇▇▇▇█▆▇█▇▇▇▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▅▆▆▇▆▆▇▇▆▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇▇█▇▆▇▇█
train_auc,▁▅▆▇▇▇▇▇▇▇▇▇▇▇██▇█▇██▇███▇█▇████████▇███
train_f1,▁▆▇▇▇▇▇▇▇█▇▇█▇███████▇████▇▇█▇██▇███▇███
train_loss_epoch,█▅▄▃▃▂▃▃▂▃▃▂▂▃▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▁▂▁▁▁▁▁▂▂▁▂
train_loss_step,▇▆▄█▃▆▅▂▂▄▅▂▃▄▅▂▃▃▃▂▄▄▂▄▆▄▄▄▄▅▃▁▃▃▅▂▄▄▃▂
trainer/global_step,▁▁▁▂▂▁▂▂▂▃▂▃▃▂▂▄▄▄▂▂▅▅▂▅▂▆▆▂▆▆▃▃▃▇▇▃▃██▃
val_acc,▁▁▆▇██▇▇▇██▆▇▆▇▆▇▇▇▇█▇▇▇▇▇▇▇▇▆▇▇▅▆▇▆▇▇▇▆
val_auc,▁▁▄▄▅▅▅▆▆▅▆▆▆▇▇▆▇▆▇▇▇▆▇▆▇▇▇▆▇▅▇▇▇▇██▇▇██
val_f1,▁▂▇███████████████▇█████████████████████


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
c:\Users\thu71\AppData\Local\anaconda3\envs\scanpy\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:617: UserWarning: Checkpoint directory \\bme-retromaster.ad.gatech.edu\labs5\coskun-lab\Thomas\23_PLA_revision\data\OCT Cell Culture\3D_Whole\saved_models\5PPI_3D\Graph_GNNs_Kfold\GINConv_norm_2_16_onehot_4\attention\GraphLevelGINConv_norm exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model       | GIN              | 1.0 K 
2  | head        | Sequential       | 154   
3  | loss_module | CrossEntropyLoss | 0     
4  | train_acc   | BinaryA

GINConv


`Trainer.fit` stopped: `max_epochs=65` reached.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▃▅▆▇▇▇▇▇▇▇▇██▇▇▇█▇█▇█▇▇▇▇▇█▇▇███▇▇█████
train_auc,▁▂▆▆▇▇▇▇▇▇▇█▇███▇███████▇███████████████
train_f1,▃▁▆▇▇▇▇▇▇▇▇▇██▇▇▇█▇█▇█▇▇▇▇▇█▇▇███▇█▇████
train_loss_epoch,█▇▄▄▃▂▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▂▁▁▂▂▁▁▁▁
train_loss_step,▆█▄▆▃▄▆▄▃▄▄▄▃▄▂▄▃▃▄▂▄▄▂▂▆▂▃▄▃▂▂▁▃▄▂▄▁▂▂▂
trainer/global_step,▁▁▁▂▂▁▂▂▂▃▂▃▃▂▂▄▄▄▂▂▅▅▂▅▂▆▆▂▆▆▃▃▃▇▇▃▃██▃
val_acc,▁▁▅▇▇▆▇▇▆▇▆▇▇█▇▇▇▇▇▇▆▆▇▇▇██▇▇▇▇▇▇▇▇▆▇▇▇▆
val_auc,▁▃▅▆▅▇▇▇▇▇▇▇▇▇██▇█▇█▇▇█▇███▇▇█▇▇▇▇▇▇▇▇▇▇
val_f1,▁▁▆██▇██▇█▇███▇█▇█▇█▇▇▇▇███▇▇█▇▇██▇▇█▇█▇
